In [50]:
# ! pip install transformers
import torch
import numpy as np
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW

from torch import nn
from transformers import BertModel
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
import torch

import pandas as pd
import numpy as np
import os

In [51]:
# df[df['content']==' ']

In [52]:
torch.cuda.empty_cache()

In [53]:
df=pd.read_csv('BERT_data.csv')
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)

In [54]:
df['cat_1_level_2']=df['cat_1_level_2']-1
df['cat_1_level_2'].unique()

array([0, 2, 3, 4, 1, 6, 5, 7], dtype=int64)

In [55]:
df=df[~(df['cat_1_level_2']==6)]
df=df[~(df['cat_1_level_2']==7)]
df['cat_1_level_2'].unique()

array([0, 2, 3, 4, 1, 5], dtype=int64)

In [56]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Dataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.labels=df['cat_1_level_2']
        self.text=[tokenizer(text,
                               padding='max_length',truncation=True,
                                return_tensors="pt") for text in df['content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [57]:
df

,message_id,subject,content,cat_1_level_2
0,<9831685.1075855725804.JavaMail.evans@thyme>,re confidenti employe informationlenhart,also need know base salari jay reitmey moniqu ...,0
1,<21041312.1075855725847.JavaMail.evans@thyme>,re person and confidenti compens inform,thank inform would help would send detail work...,2
2,<21261996.1075858638025.JavaMail.evans@thyme>,re western wholesal activ gas power conf call ...,guy coordin make sure someon listen confer cal...,3
3,<20399547.1075857614321.JavaMail.evans@thyme>,re confidenti employe informationdutch quigley,thx jeani slone pm john cc confidenti employe ...,4
4,<860767.1075849626951.JavaMail.evans@thyme>,re hp confidenti intern document,excel updat thank put togeth dalepatrick let r...,0
...,...,...,...,...
1485,<18361957.1075861368310.JavaMail.evans@thyme>,confidenti due dilig info g,lot confidenti inform item detail month pl cov...,0
1486,<29049822.1075842045435.JavaMail.evans@thyme>,california dim bulb,may seen first came back worth rerun,0
1487,<205897.1075861997314.JavaMail.evans@thyme>,re confidenti ee info,mark email fax ani written document give heidi...,4
1488,<23743848.1075863311776.JavaMail.evans@thyme>,confidenti inform and secur trade,william william enron wholesal servic offic ch...,0


In [58]:
df.reset_index(inplace=True)
df=df.drop('index',axis=1)
len(df)

1458

In [59]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.3):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 6)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer
model = BertClassifier()

In [61]:
n=0
for x in model.state_dict():
    n=n+1
    print(x)
n

'\nn=0\nfor x in model.state_dict():\n    n=n+1\n    print(x)\nn\n'

In [62]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [63]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=5)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df)

KeyboardInterrupt: 